<a href="https://colab.research.google.com/github/abhaydubey200/card_memo/blob/main/final_all_models.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
df1=pd.read_excel('Small Cardamom Auction Average Price 2014-2025..xlsx')
df1.head()

,Date of Auction,Qty Sold (Kgs),Avg.Price (Rs./Kg)
0,2025-10-29,101810.3,2503.80
1,2025-10-29,21767.9,2467.68
2,2025-10-28,84888.1,2464.22
3,2025-10-28,90880.9,2479.12
4,2025-10-27,79737.1,2466.66


In [ ]:
df2=pd.read_excel('data (32)2.xlsx')
df2.head()

,Year,Jan.,Feb.,Mar.,Apr.,May,Jun.,Jul.,Aug.,Sept.,Oct.,Nov.,Dec.,Annual
0,1901,28.7,44.7,51.6,159.9,174.7,824.6,743.0,357.5,197.7,266.9,350.8,48.4,3248.5
1,1902,6.7,2.6,57.3,83.9,134.5,390.9,1205.0,315.8,491.6,358.4,158.3,121.5,3326.5
2,1903,3.1,18.6,3.1,83.6,249.7,558.6,1022.5,420.2,341.8,354.0,157.0,59.0,3271.1
3,1904,23.7,3.0,32.2,71.5,235.7,1098.2,725.5,351.8,222.7,328.1,33.9,3.3,3129.6
4,1905,1.2,22.3,9.3,105.9,263.3,850.2,520.5,293.6,217.2,383.5,74.4,0.2,2741.5


In [ ]:
df1.columns

Index(['Date of Auction', 'Qty Sold (Kgs)', 'Avg.Price (Rs./Kg)'], dtype='object')

In [ ]:
df2.columns

Index(['Year', 'Jan.', 'Feb.', 'Mar.', 'Apr.', 'May', 'Jun.', 'Jul.', 'Aug.',
       'Sept.', 'Oct.', 'Nov.', 'Dec.', 'Annual'],
      dtype='object')

In [ ]:
import pandas as pd

df1['Date of Auction'] = pd.to_datetime(df1['Date of Auction'])

df1['year'] = df1['Date of Auction'].dt.year
df1['month'] = df1['Date of Auction'].dt.month


In [ ]:
df1_monthly = (
    df1.groupby(['year', 'month'], as_index=False)
       .agg({
           'Qty Sold (Kgs)': 'sum',          # ✅ SUM
           'Avg.Price (Rs./Kg)': 'mean'      # avg price
       })
)

df1_monthly.rename(columns={
    'Qty Sold (Kgs)': 'quantity_sold',
    'Avg.Price (Rs./Kg)': 'avg_price'
}, inplace=True)


In [ ]:
df2_clean = df2.drop(columns=['Annual'])


In [ ]:
df2_long = df2_clean.melt(
    id_vars='Year',
    var_name='month',
    value_name='quantity_sold'
)


In [ ]:
month_map = {
    'Jan.':1, 'Feb.':2, 'Mar.':3, 'Apr.':4,
    'May':5, 'Jun.':6, 'Jul.':7, 'Aug.':8,
    'Sept.':9, 'Oct.':10, 'Nov.':11, 'Dec.':12
}

df2_long['month'] = df2_long['month'].map(month_map)
df2_long.rename(columns={'Year':'year'}, inplace=True)


In [ ]:
final_df = pd.merge(
    df2_long,
    df1_monthly[['year','month','avg_price']],
    on=['year','month'],
    how='left'
)


In [ ]:
final_df['avg_price'].fillna(
    final_df['avg_price'].median(),
    inplace=True
)


/tmp/ipython-input-4287227175.py:1: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  final_df['avg_price'].fillna(


In [ ]:
print(final_df.head())


   year  month  quantity_sold    avg_price
0  1901      1           28.7  1250.034762
1  1902      1            6.7  1250.034762
2  1903      1            3.1  1250.034762
3  1904      1           23.7  1250.034762
4  1905      1            1.2  1250.034762


In [ ]:
final_df = final_df.sort_values(['year','month'])

final_df['lag_1'] = final_df['quantity_sold'].shift(1)
final_df['lag_3'] = final_df['quantity_sold'].shift(3)
final_df['lag_6'] = final_df['quantity_sold'].shift(6)

final_df['roll_mean_3'] = final_df['quantity_sold'].rolling(3).mean()

final_df.dropna(inplace=True)


In [ ]:
from sklearn.ensemble import RandomForestRegressor

features = [
    'year','month',
    'lag_1','lag_3','lag_6',
    'roll_mean_3',
    'avg_price'
]

X = final_df[features]
y = final_df['quantity_sold']

model = RandomForestRegressor(
    n_estimators=300,
    max_depth=15,
    random_state=42
)

model.fit(X, y)


RandomForestRegressor(max_depth=15, n_estimators=300, random_state=42)

In [ ]:
print("Avg monthly qty:", final_df['quantity_sold'].mean())
print("2-year rough estimate:", final_df['quantity_sold'].mean() * 24)


Avg monthly qty: 243.4407109322602
2-year rough estimate: 5842.577062374245


In [ ]:
final_df = final_df.sort_values(['year', 'month']).reset_index(drop=True)

history = final_df.copy()


In [ ]:
def create_future_months(last_year, last_month, periods=24):
    future = []
    year, month = last_year, last_month

    for _ in range(periods):
        month += 1
        if month > 12:
            month = 1
            year += 1
        future.append({'year': year, 'month': month})

    return pd.DataFrame(future)


In [ ]:
last_year = history.iloc[-1]['year']
last_month = history.iloc[-1]['month']

future_months = create_future_months(last_year, last_month, 24)


In [ ]:
future_predictions = []

for i in range(24):

    lag_1 = history.iloc[-1]['quantity_sold']
    lag_3 = history.iloc[-3]['quantity_sold']
    lag_6 = history.iloc[-6]['quantity_sold']
    roll_mean_3 = history.tail(3)['quantity_sold'].mean()

    # keep price stable (or update if you have future price logic)
    avg_price = history.iloc[-1]['avg_price']

    X_future = pd.DataFrame([{
        'year': future_months.iloc[i]['year'],
        'month': future_months.iloc[i]['month'],
        'lag_1': lag_1,
        'lag_3': lag_3,
        'lag_6': lag_6,
        'roll_mean_3': roll_mean_3,
        'avg_price': avg_price
    }])

    pred_qty = model.predict(X_future)[0]

    future_predictions.append({
        'year': future_months.iloc[i]['year'],
        'month': future_months.iloc[i]['month'],
        'predicted_quantity': pred_qty
    })

    # append prediction back to history
    history = pd.concat([
        history,
        pd.DataFrame([{
            'year': future_months.iloc[i]['year'],
            'month': future_months.iloc[i]['month'],
            'quantity_sold': pred_qty,
            'avg_price': avg_price
        }])
    ], ignore_index=True)


In [ ]:
forecast_24m = pd.DataFrame(future_predictions)
print(forecast_24m)


    year  month  predicted_quantity
0   2025     10          323.132927
1   2025     11          194.846039
2   2025     12           97.502000
3   2026      1           39.027333
4   2026      2           35.886827
5   2026      3           81.684000
6   2026      4           63.738667
7   2026      5          120.208892
8   2026      6          112.316635
9   2026      7          129.107208
10  2026      8          164.814167
11  2026      9          141.220897
12  2026     10          168.933167
13  2026     11          146.588085
14  2026     12           90.060133
15  2027      1           38.981667
16  2027      2           32.388417
17  2027      3           81.195000
18  2027      4           64.176667
19  2027      5          120.083714
20  2027      6          112.431302
21  2027      7          127.858667
22  2027      8          164.976167
23  2027      9          142.799230


In [ ]:
final_df['log_qty'] = np.log1p(final_df['quantity_sold'])


In [ ]:
X = final_df[features]
y = final_df['log_qty']

rf_model = RandomForestRegressor(
    n_estimators=500,
    max_depth=18,
    min_samples_split=2,
    random_state=42
)

rf_model.fit(X, y)


RandomForestRegressor(max_depth=18, n_estimators=500, random_state=42)

In [ ]:
log_pred = rf_model.predict(X_future)[0]

# 🔥 THIS LINE CHANGES EVERYTHING
pred_qty = np.expm1(log_pred)


In [ ]:
print("Historical Avg:", final_df['quantity_sold'].mean())
print("Forecast Avg:", forecast_24m['predicted_quantity'].mean())


Historical Avg: 243.4407109322602
Forecast Avg: 116.41490858729401


In [ ]:
final_df['time_index'] = range(len(final_df))


In [ ]:
features.append('time_index')


In [ ]:
import pandas as pd
import numpy as np

from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error, r2_score


In [ ]:
df1['Date of Auction'] = pd.to_datetime(df1['Date of Auction'])

df1['year'] = df1['Date of Auction'].dt.year
df1['month'] = df1['Date of Auction'].dt.month

df1_monthly = (
    df1.groupby(['year', 'month'], as_index=False)
       .agg({
           'Qty Sold (Kgs)': 'sum',
           'Avg.Price (Rs./Kg)': 'mean'
       })
)

df1_monthly.rename(columns={
    'Qty Sold (Kgs)': 'quantity_sold',
    'Avg.Price (Rs./Kg)': 'avg_price'
}, inplace=True)


In [ ]:
df2_clean = df2.drop(columns=['Annual'])

df2_long = df2_clean.melt(
    id_vars='Year',
    var_name='month',
    value_name='quantity_sold'
)

month_map = {
    'Jan.':1, 'Feb.':2, 'Mar.':3, 'Apr.':4,
    'May':5, 'Jun.':6, 'Jul.':7, 'Aug.':8,
    'Sept.':9, 'Oct.':10, 'Nov.':11, 'Dec.':12
}

df2_long['month'] = df2_long['month'].map(month_map)
df2_long.rename(columns={'Year':'year'}, inplace=True)


In [ ]:
final_df = pd.merge(
    df2_long,
    df1_monthly[['year','month','avg_price']],
    on=['year','month'],
    how='left'
)

final_df['avg_price'].fillna(final_df['avg_price'].median(), inplace=True)


/tmp/ipython-input-3606567185.py:8: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  final_df['avg_price'].fillna(final_df['avg_price'].median(), inplace=True)


In [34]:
final_df = final_df.sort_values(['year','month']).reset_index(drop=True)
final_df['time_index'] = range(len(final_df))


In [35]:
final_df['lag_1'] = final_df['quantity_sold'].shift(1)
final_df['lag_3'] = final_df['quantity_sold'].shift(3)
final_df['lag_6'] = final_df['quantity_sold'].shift(6)

final_df['roll_mean_3'] = final_df['quantity_sold'].rolling(3).mean()

final_df.dropna(inplace=True)


In [36]:
final_df['log_qty'] = np.log1p(final_df['quantity_sold'])


In [37]:
features = [
    'year', 'month', 'time_index',
    'lag_1', 'lag_3', 'lag_6',
    'roll_mean_3',
    'avg_price'
]

X = final_df[features]
y = final_df['log_qty']

rf_model = RandomForestRegressor(
    n_estimators=500,
    max_depth=18,
    min_samples_split=2,
    min_samples_leaf=1,
    random_state=42,
    n_jobs=-1
)

rf_model.fit(X, y)


RandomForestRegressor(max_depth=18, n_estimators=500, n_jobs=-1,
                      random_state=42)

In [38]:
def create_future_months(last_year, last_month, start_index, periods=24):
    future = []
    year, month = last_year, last_month
    idx = start_index

    for _ in range(periods):
        month += 1
        idx += 1
        if month > 12:
            month = 1
            year += 1
        future.append({'year': year, 'month': month, 'time_index': idx})

    return pd.DataFrame(future)


In [39]:
history = final_df.copy()

future_months = create_future_months(
    history.iloc[-1]['year'],
    history.iloc[-1]['month'],
    history.iloc[-1]['time_index'],
    24
)

forecast = []

for i in range(24):

    lag_1 = history.iloc[-1]['quantity_sold']
    lag_3 = history.iloc[-3]['quantity_sold']
    lag_6 = history.iloc[-6]['quantity_sold']
    roll_mean_3 = history.tail(3)['quantity_sold'].mean()
    avg_price = history.iloc[-1]['avg_price']

    X_future = pd.DataFrame([{
        'year': future_months.iloc[i]['year'],
        'month': future_months.iloc[i]['month'],
        'time_index': future_months.iloc[i]['time_index'],
        'lag_1': lag_1,
        'lag_3': lag_3,
        'lag_6': lag_6,
        'roll_mean_3': roll_mean_3,
        'avg_price': avg_price
    }])

    log_pred = rf_model.predict(X_future)[0]
    pred_qty = np.expm1(log_pred)   # 🔥 SCALE FIX

    forecast.append({
        'year': future_months.iloc[i]['year'],
        'month': future_months.iloc[i]['month'],
        'predicted_quantity': pred_qty
    })

    history = pd.concat([
        history,
        pd.DataFrame([{
            'year': future_months.iloc[i]['year'],
            'month': future_months.iloc[i]['month'],
            'time_index': future_months.iloc[i]['time_index'],
            'quantity_sold': pred_qty,
            'avg_price': avg_price
        }])
    ], ignore_index=True)


In [40]:
forecast_24m = pd.DataFrame(forecast)
print(forecast_24m)


    year  month  predicted_quantity
0   2025     10          300.541553
1   2025     11          196.935414
2   2025     12           39.019252
3   2026      1           13.408425
4   2026      2           10.495599
5   2026      3           32.601639
6   2026      4           39.586155
7   2026      5           51.037165
8   2026      6           60.450371
9   2026      7           77.763237
10  2026      8           96.285670
11  2026      9          114.330221
12  2026     10          118.175028
13  2026     11          107.742804
14  2026     12           24.082874
15  2027      1            8.107663
16  2027      2            5.497517
17  2027      3           22.946293
18  2027      4           33.242262
19  2027      5           35.559815
20  2027      6           57.671404
21  2027      7           62.728023
22  2027      8           81.375315
23  2027      9           98.871950


In [41]:
total_2yr_qty = int(forecast_24m['predicted_quantity'].sum())
print("🔥 Expected Quantity for Next 24 Months:", total_2yr_qty)


🔥 Expected Quantity for Next 24 Months: 1688


In [42]:
print("Historical Monthly Avg:", final_df['quantity_sold'].mean())
print("Forecast Monthly Avg:", forecast_24m['predicted_quantity'].mean())


Historical Monthly Avg: 243.4407109322602
Forecast Monthly Avg: 70.35231873262363


In [43]:
final_df = final_df.sort_values(['year','month']).reset_index(drop=True)

# Lags for quantity
final_df['qty_lag_1'] = final_df['quantity_sold'].shift(1)
final_df['qty_lag_3'] = final_df['quantity_sold'].shift(3)
final_df['qty_lag_6'] = final_df['quantity_sold'].shift(6)

final_df['qty_roll_3'] = final_df['quantity_sold'].rolling(3).mean()

# Lags for price
final_df['price_lag_1'] = final_df['avg_price'].shift(1)
final_df['price_lag_3'] = final_df['avg_price'].shift(3)

final_df.dropna(inplace=True)


In [44]:
final_df['log_qty'] = np.log1p(final_df['quantity_sold'])


In [45]:
from sklearn.ensemble import RandomForestRegressor

qty_features = [
    'year','month','time_index',
    'qty_lag_1','qty_lag_3','qty_lag_6',
    'qty_roll_3'
]

X_qty = final_df[qty_features]
y_qty = final_df['log_qty']

rf_qty = RandomForestRegressor(
    n_estimators=500,
    max_depth=18,
    random_state=42,
    n_jobs=-1
)

rf_qty.fit(X_qty, y_qty)


RandomForestRegressor(max_depth=18, n_estimators=500, n_jobs=-1,
                      random_state=42)

In [46]:
price_features = [
    'year','month','time_index',
    'price_lag_1','price_lag_3'
]

X_price = final_df[price_features]
y_price = final_df['avg_price']

rf_price = RandomForestRegressor(
    n_estimators=300,
    max_depth=12,
    random_state=42,
    n_jobs=-1
)

rf_price.fit(X_price, y_price)


RandomForestRegressor(max_depth=12, n_estimators=300, n_jobs=-1,
                      random_state=42)

In [47]:
def future_dates(last_year, last_month, last_index, periods=24):
    data = []
    y, m, idx = last_year, last_month, last_index

    for _ in range(periods):
        m += 1
        idx += 1
        if m > 12:
            m = 1
            y += 1
        data.append({'year': y, 'month': m, 'time_index': idx})

    return pd.DataFrame(data)


In [48]:
history = final_df.copy()

future_df = future_dates(
    history.iloc[-1]['year'],
    history.iloc[-1]['month'],
    history.iloc[-1]['time_index'],
    24
)

forecast = []

for i in range(24):

    # ---------- Quantity features ----------
    X_qty_future = pd.DataFrame([{
        'year': future_df.iloc[i]['year'],
        'month': future_df.iloc[i]['month'],
        'time_index': future_df.iloc[i]['time_index'],
        'qty_lag_1': history.iloc[-1]['quantity_sold'],
        'qty_lag_3': history.iloc[-3]['quantity_sold'],
        'qty_lag_6': history.iloc[-6]['quantity_sold'],
        'qty_roll_3': history.tail(3)['quantity_sold'].mean()
    }])

    log_qty_pred = rf_qty.predict(X_qty_future)[0]
    qty_pred = np.expm1(log_qty_pred)   # 🔥 scale restored

    # ---------- Price features ----------
    X_price_future = pd.DataFrame([{
        'year': future_df.iloc[i]['year'],
        'month': future_df.iloc[i]['month'],
        'time_index': future_df.iloc[i]['time_index'],
        'price_lag_1': history.iloc[-1]['avg_price'],
        'price_lag_3': history.iloc[-3]['avg_price']
    }])

    price_pred = rf_price.predict(X_price_future)[0]

    forecast.append({
        'date': f"{future_df.iloc[i]['year']}-{future_df.iloc[i]['month']:02d}-01",
        'predicted_quantity': qty_pred,
        'predicted_price': price_pred
    })

    # Append to history
    history = pd.concat([
        history,
        pd.DataFrame([{
            'year': future_df.iloc[i]['year'],
            'month': future_df.iloc[i]['month'],
            'time_index': future_df.iloc[i]['time_index'],
            'quantity_sold': qty_pred,
            'avg_price': price_pred
        }])
    ], ignore_index=True)


In [49]:
forecast_24m = pd.DataFrame(forecast)
forecast_24m['date'] = pd.to_datetime(forecast_24m['date'])

print(forecast_24m)


         date  predicted_quantity  predicted_price
0  2025-10-01          325.356310      2505.571767
1  2025-11-01          190.858563      2585.903980
2  2025-12-01           46.634434      2611.936171
3  2026-01-01           15.591881      2479.311338
4  2026-02-01            7.618021      2465.906527
5  2026-03-01           28.394464      2416.538138
6  2026-04-01           36.739765      2419.791560
7  2026-05-01           44.741858      2504.213923
8  2026-06-01           62.970285      2490.344077
9  2026-07-01           74.484729      2509.863403
10 2026-08-01          111.415958      2500.729762
11 2026-09-01          107.486152      2495.351223
12 2026-10-01          130.490604      2513.488812
13 2026-11-01          112.259235      2585.903980
14 2026-12-01           25.276313      2610.516564
15 2027-01-01            6.489602      2479.311338
16 2027-02-01            5.365436      2465.906527
17 2027-03-01           21.949021      2416.538138
18 2027-04-01           33.8960

In [52]:
forecast_24m.to_excel(
    "cardamom_24_month_forecast.xlsx",
    index=False
)

print("✅ Forecast saved as cardamom_24_month_forecast.csv")


✅ Forecast saved as cardamom_24_month_forecast.csv


In [53]:
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestRegressor

# Convert date
df1['Date of Auction'] = pd.to_datetime(df1['Date of Auction'])

# Aggregate to MONTHLY SUM
monthly = df1.groupby(pd.Grouper(key='Date of Auction', freq='MS')).agg({
    'Qty Sold (Kgs)': 'sum',         # Monthly total quantity
    'Avg.Price (Rs./Kg)': 'mean'     # Average price for the month
}).reset_index()

monthly.rename(columns={
    'Date of Auction': 'date',
    'Qty Sold (Kgs)': 'monthly_qty',
    'Avg.Price (Rs./Kg)': 'avg_price'
}, inplace=True)


In [54]:
monthly['year'] = monthly['date'].dt.year
monthly['month'] = monthly['date'].dt.month

# Encode seasonality for better RF performance
monthly['month_sin'] = np.sin(2 * np.pi * monthly['month'] / 12)
monthly['month_cos'] = np.cos(2 * np.pi * monthly['month'] / 12)

# Log-transform quantity for scale
monthly['log_qty'] = np.log1p(monthly['monthly_qty'])


In [55]:
qty_features = ['year', 'month', 'month_sin', 'month_cos']
X_qty = monthly[qty_features]
y_qty = monthly['log_qty']

rf_qty = RandomForestRegressor(
    n_estimators=500,
    max_depth=12,
    random_state=42,
    n_jobs=-1
)
rf_qty.fit(X_qty, y_qty)


RandomForestRegressor(max_depth=12, n_estimators=500, n_jobs=-1,
                      random_state=42)

In [57]:
price_features = ['year', 'month', 'month_sin', 'month_cos']

# Impute NaN values in avg_price before creating X_price and y_price
monthly['avg_price'].fillna(monthly['avg_price'].median(), inplace=True)

X_price = monthly[price_features]
y_price = monthly['avg_price']

rf_price = RandomForestRegressor(
    n_estimators=300,
    max_depth=10,
    random_state=42
)
rf_price.fit(X_price, y_price)

RandomForestRegressor(max_depth=10, n_estimators=300, random_state=42)

In [58]:
future_dates = pd.date_range(
    start=monthly['date'].max() + pd.offsets.MonthBegin(1),
    periods=24,
    freq='MS'
)

future = pd.DataFrame({'date': future_dates})
future['year'] = future['date'].dt.year
future['month'] = future['date'].dt.month
future['month_sin'] = np.sin(2 * np.pi * future['month'] / 12)
future['month_cos'] = np.cos(2 * np.pi * future['month'] / 12)


In [59]:
# Predict log quantity → convert back
future['predicted_quantity'] = np.expm1(
    rf_qty.predict(future[qty_features])
)

# Predict price
future['predicted_price'] = rf_price.predict(future[price_features])

# Round values for clarity
future['predicted_quantity'] = future['predicted_quantity'].round(0).astype(int)
future['predicted_price'] = future['predicted_price'].round(2)


In [79]:
#future.to_csv("cardamom_rf_24_month_forecast.csv", index=False)
future.to_excel("cardamom_rf_24_month_forecast.xlsx", index=False)


In [63]:
import pandas as pd
from prophet import Prophet
import numpy as np

# Convert date
df1['Date of Auction'] = pd.to_datetime(df1['Date of Auction'])

# Aggregate monthly
monthly = df1.groupby(pd.Grouper(key='Date of Auction', freq='MS')).agg({
    'Qty Sold (Kgs)': 'sum',
    'Avg.Price (Rs./Kg)': 'mean'
}).reset_index()

monthly.rename(columns={
    'Date of Auction': 'ds',
    'Qty Sold (Kgs)': 'monthly_qty',
    'Avg.Price (Rs./Kg)': 'avg_price'
}, inplace=True)

# Prophet requires 'y' as target
monthly['y'] = np.log1p(monthly['monthly_qty'])  # log transform for large numbers

# Fit Prophet model
qty_model = Prophet(yearly_seasonality=True, daily_seasonality=False, weekly_seasonality=False)
qty_model.fit(monthly[['ds', 'y']])

# Create future dataframe for next 24 months
future_qty = qty_model.make_future_dataframe(periods=24, freq='MS')

# Predict
forecast_qty = qty_model.predict(future_qty)

# Back-transform log scale
forecast_qty['predicted_quantity'] = np.expm1(forecast_qty['yhat']).round(0).astype(int)


In [64]:
monthly['y'] = monthly['avg_price']  # no log transform for price

price_model = Prophet(yearly_seasonality=True, daily_seasonality=False, weekly_seasonality=False)
price_model.fit(monthly[['ds', 'y']])

future_price = price_model.make_future_dataframe(periods=24, freq='MS')
forecast_price = price_model.predict(future_price)

forecast_price['predicted_price'] = forecast_price['yhat'].round(2)


In [69]:
# Step 1: Prepare monthly data
monthly['ds'] = monthly['ds']  # ensure date column exists
monthly['y'] = monthly['avg_price']  # target column must be 'y'

# Step 2: Initialize and fit Prophet
from prophet import Prophet
price_model = Prophet(yearly_seasonality=True, daily_seasonality=False, weekly_seasonality=False)
price_model.fit(monthly[['ds', 'y']])  # MUST use 'ds' and 'y'

# Step 3: Create future dataframe for 24 months
future_price = price_model.make_future_dataframe(periods=24, freq='MS')

# Step 4: Predict
forecast_price = price_model.predict(future_price)

# Step 5: Rename predicted column
forecast_price['predicted_price'] = forecast_price['yhat'].round(2)


In [66]:
# Create the target column 'y' for Prophet
monthly['y'] = monthly['avg_price']  # price does NOT need log transform

# Fit Prophet model
price_model = Prophet(yearly_seasonality=True, daily_seasonality=False, weekly_seasonality=False)
price_model.fit(monthly[['ds', 'y']])  # MUST use 'y' column

# Create future dataframe for next 24 months
future_price = price_model.make_future_dataframe(periods=24, freq='MS')

# Predict
forecast_price = price_model.predict(future_price)

# Rename forecast column for clarity
forecast_price['predicted_price'] = forecast_price['yhat'].round(2)


In [70]:
future_qty = qty_model.make_future_dataframe(periods=24, freq='MS')
future_price = price_model.make_future_dataframe(periods=24, freq='MS')


In [71]:
forecast_qty = qty_model.predict(future_qty)
forecast_qty['predicted_quantity'] = np.expm1(forecast_qty['yhat']).round(0).astype(int)


In [72]:
forecast_price = price_model.predict(future_price)
forecast_price['predicted_price'] = forecast_price['yhat'].round(2)


In [73]:
future_forecast = pd.DataFrame({
    'date': forecast_qty['ds'],
    'predicted_quantity': forecast_qty['predicted_quantity'],
    'predicted_price': forecast_price['predicted_price']
})

# Keep only the next 24 months
future_forecast = future_forecast.tail(24)


In [74]:
future_forecast.to_csv("cardamom_prophet_24_month_forecast.csv", index=False)
future_forecast.to_excel("cardamom_prophet_24_month_forecast.xlsx", index=False)


In [75]:
# Aggregate monthly
monthly = df1.groupby(pd.Grouper(key='Date of Auction', freq='MS')).agg({
    'Qty Sold (Kgs)': 'sum',
    'Avg.Price (Rs./Kg)': 'mean'
}).reset_index()

monthly.rename(columns={
    'Date of Auction': 'ds',
    'Qty Sold (Kgs)': 'y',  # Prophet target
    'Avg.Price (Rs./Kg)': 'avg_price'
}, inplace=True)

# Fit Prophet on quantity directly (no log)
qty_model = Prophet(yearly_seasonality=True, daily_seasonality=False, weekly_seasonality=False)
qty_model.fit(monthly[['ds','y']])

# Create future dataframe
future_qty = qty_model.make_future_dataframe(periods=24, freq='MS')

# Predict
forecast_qty = qty_model.predict(future_qty)

# Keep predicted quantity directly (no expm1)
forecast_qty['predicted_quantity'] = forecast_qty['yhat'].round(0).astype(int)


In [76]:
import pandas as pd
from prophet import Prophet

# -------------------------------
# 1️⃣ Prepare Monthly Aggregated Data
# -------------------------------
df1['Date of Auction'] = pd.to_datetime(df1['Date of Auction'])

monthly = df1.groupby(pd.Grouper(key='Date of Auction', freq='MS')).agg({
    'Qty Sold (Kgs)': 'sum',         # monthly total quantity
    'Avg.Price (Rs./Kg)': 'mean'     # monthly average price
}).reset_index()

# Rename columns for Prophet
monthly.rename(columns={
    'Date of Auction': 'ds',
    'Qty Sold (Kgs)': 'y',           # target for quantity
    'Avg.Price (Rs./Kg)': 'avg_price'
}, inplace=True)

# -------------------------------
# 2️⃣ Forecast Quantity
# -------------------------------
qty_model = Prophet(yearly_seasonality=True, daily_seasonality=False, weekly_seasonality=False)
qty_model.fit(monthly[['ds','y']])

future_qty = qty_model.make_future_dataframe(periods=24, freq='MS')
forecast_qty = qty_model.predict(future_qty)

# Correct scale: no log-transform
forecast_qty['predicted_quantity'] = forecast_qty['yhat'].round(0).astype(int)

# -------------------------------
# 3️⃣ Forecast Price
# -------------------------------
monthly_price = monthly[['ds', 'avg_price']].rename(columns={'avg_price':'y'})  # Prophet target must be 'y'

price_model = Prophet(yearly_seasonality=True, daily_seasonality=False, weekly_seasonality=False)
price_model.fit(monthly_price)

future_price = price_model.make_future_dataframe(periods=24, freq='MS')
forecast_price = price_model.predict(future_price)
forecast_price['predicted_price'] = forecast_price['yhat'].round(2)

# -------------------------------
# 4️⃣ Combine Forecasts
# -------------------------------
future_forecast = pd.DataFrame({
    'date': forecast_qty['ds'],
    'predicted_quantity': forecast_qty['predicted_quantity'],
    'predicted_price': forecast_price['predicted_price']
})

# Keep only next 24 months
future_forecast = future_forecast.tail(24)

# -------------------------------
# 5️⃣ Save Forecast to CSV / Excel
# -------------------------------
future_forecast.to_csv("cardamom_prophet_24_month_forecast.csv", index=False)
future_forecast.to_excel("cardamom_prophet_24_month_forecast.xlsx", index=False)

# -------------------------------
# 6️⃣ Show Result
# -------------------------------
print(future_forecast)


          date  predicted_quantity  predicted_price
130 2025-11-01             2837745          2180.54
131 2025-12-01             3027783          2260.21
132 2026-01-01             2564562          2367.16
133 2026-02-01             2357828          2306.97
134 2026-03-01             2175663          2194.05
135 2026-04-01             2044452          2078.79
136 2026-05-01             1994213          2119.03
137 2026-06-01             1759061          2213.90
138 2026-07-01             2035120          2325.27
139 2026-08-01             2527701          2399.23
140 2026-09-01             2548279          2402.25
141 2026-10-01             2916630          2272.38
142 2026-11-01             2909406          2315.00
143 2026-12-01             3118396          2396.47
144 2027-01-01             2642711          2503.39
145 2027-02-01             2435823          2442.02
146 2027-03-01             2227616          2325.86
147 2027-04-01             2087744          2212.83
148 2027-05-

In [77]:
# 5️⃣ Save Forecast to CSV / Excel
# -------------------------------
future_forecast.to_csv("cardamom_prophet_24_month_forecast.csv", index=False)
future_forecast.to_excel("cardamom_prophet_24_month_forecast.xlsx", index=False)


In [95]:
!pip install lightgbm --quiet


In [96]:
import pandas as pd
import numpy as np
import lightgbm as lgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error


In [97]:
# Aggregate monthly
df1['Date of Auction'] = pd.to_datetime(df1['Date of Auction'])

monthly = df1.groupby(pd.Grouper(key='Date of Auction', freq='MS')).agg({
    'Qty Sold (Kgs)': 'sum',
    'Avg.Price (Rs./Kg)': 'mean'
}).reset_index()

monthly.rename(columns={
    'Date of Auction': 'date',
    'Qty Sold (Kgs)': 'quantity',
    'Avg.Price (Rs./Kg)': 'price'
}, inplace=True)

# Add time features
monthly['month'] = monthly['date'].dt.month
monthly['year'] = monthly['date'].dt.year
monthly['month_sin'] = np.sin(2 * np.pi * monthly['month'] / 12)
monthly['month_cos'] = np.cos(2 * np.pi * monthly['month'] / 12)


In [98]:
# Lag features for quantity
for lag in range(1, 13):  # past 12 months
    monthly[f'qty_lag_{lag}'] = monthly['quantity'].shift(lag)
    monthly[f'price_lag_{lag}'] = monthly['price'].shift(lag)

# Rolling mean features
monthly['qty_roll_mean_3'] = monthly['quantity'].shift(1).rolling(3).mean()
monthly['price_roll_mean_3'] = monthly['price'].shift(1).rolling(3).mean()

# Drop rows with NaN from lagging
monthly = monthly.dropna().reset_index(drop=True)


In [99]:
qty_features = ['month','year','month_sin','month_cos'] + \
               [f'qty_lag_{i}' for i in range(1,13)] + ['qty_roll_mean_3']
price_features = ['month','year','month_sin','month_cos'] + \
                 [f'price_lag_{i}' for i in range(1,13)] + ['price_roll_mean_3']

X_qty = monthly[qty_features]
y_qty = monthly['quantity']

X_price = monthly[price_features]
y_price = monthly['price']


In [100]:
# Quantity model
qty_model = lgb.LGBMRegressor(
    n_estimators=1000,
    learning_rate=0.05,
    max_depth=6
)
qty_model.fit(X_qty, y_qty)

# Price model
price_model = lgb.LGBMRegressor(
    n_estimators=1000,
    learning_rate=0.05,
    max_depth=6
)
price_model.fit(X_price, y_price)


[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000870 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 475
[LightGBM] [Info] Number of data points in the train set: 96, number of used features: 17
[LightGBM] [Info] Start training from score 2081154.446289
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gai

LGBMRegressor(learning_rate=0.05, max_depth=6, n_estimators=1000)

In [102]:
future_qty_preds = []
future_price_preds = []

# Copy of last known for rolling
last_known_qty = monthly.copy()
last_known_price = monthly.copy()

for date in future_dates:
    month = date.month
    year = date.year
    month_sin = np.sin(2 * np.pi * month / 12)
    month_cos = np.cos(2 * np.pi * month / 12)

    # Prepare lag features for quantity
    qty_lags = last_known_qty['quantity'].values[-12:]
    qty_roll = last_known_qty['quantity'].values[-3:].mean()

    X_new_qty = pd.DataFrame([ [month, year, month_sin, month_cos] + list(qty_lags) + [qty_roll] ],
                            columns=qty_features)

    qty_pred = qty_model.predict(X_new_qty)[0]
    future_qty_preds.append(int(round(qty_pred,0)))

    # Update last_known_qty using pd.concat
    new_row = pd.DataFrame({'date':[date], 'quantity':[qty_pred], 'price':[0],
                            'month':[month], 'year':[year], 'month_sin':[month_sin], 'month_cos':[month_cos]})
    last_known_qty = pd.concat([last_known_qty, new_row], ignore_index=True)

# Same for price
for date in future_dates:
    month = date.month
    year = date.year
    month_sin = np.sin(2 * np.pi * month / 12)
    month_cos = np.cos(2 * np.pi * month / 12)

    price_lags = last_known_price['price'].values[-12:]
    price_roll = last_known_price['price'].values[-3:].mean()

    X_new_price = pd.DataFrame([ [month, year, month_sin, month_cos] + list(price_lags) + [price_roll] ],
                               columns=price_features)

    price_pred = price_model.predict(X_new_price)[0]
    future_price_preds.append(round(price_pred,2))

    new_row = pd.DataFrame({'date':[date], 'quantity':[0], 'price':[price_pred],
                            'month':[month], 'year':[year], 'month_sin':[month_sin], 'month_cos':[month_cos]})
    last_known_price = pd.concat([last_known_price, new_row], ignore_index=True)


In [103]:
future_forecast = pd.DataFrame({
    'date': future_dates,
    'predicted_quantity': future_qty_preds,
    'predicted_price': future_price_preds
})

future_forecast.to_csv("cardamom_lgbm_24_month_forecast.csv", index=False)
future_forecast.to_excel("cardamom_lgbm_24_month_forecast.xlsx", index=False)

print(future_forecast)


         date  predicted_quantity  predicted_price
0  2025-11-01             2506807          2812.91
1  2025-12-01             2811527          2849.38
2  2026-01-01             2092131          2909.09
3  2026-02-01             1501031          2690.36
4  2026-03-01             1165673          2522.14
5  2026-04-01             1986044          2305.81
6  2026-05-01             1446615          2278.60
7  2026-06-01             1563757          2605.43
8  2026-07-01             2300587          2605.43
9  2026-08-01             3221944          2500.76
10 2026-09-01             3538164          2495.29
11 2026-10-01             2927136          2584.73
12 2026-11-01             2734533          2812.91
13 2026-12-01             2686741          2849.38
14 2027-01-01             1867077          2909.09
15 2027-02-01             1104009          2690.36
16 2027-03-01             1112465          2522.14
17 2027-04-01             1995386          2278.60
18 2027-05-01             13117